KPIs are undecided, therefore we prepare code with the "TargetVariable".

### Split Data in Training, Validation and Test set

In [1]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("data/hourly_utilization_site_1.csv")

### Encoding

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
def encode_non_numeric_columns(df):
    # Make a copy of the DataFrame to avoid modifying the original
    df_encoded = df.copy()
    
    # Iterate through each column
    for col in df_encoded.columns:
        # Check if the column is non-numeric
        if df_encoded[col].dtype == 'object' or df_encoded[col].dtype.name == 'category':
            # Apply LabelEncoder to the column
            le = LabelEncoder()
            df_encoded[col] = le.fit_transform(df_encoded[col])
    
    return df_encoded

In [7]:
df_encoded = encode_non_numeric_columns(df)

In [9]:
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)

In [ ]:
train_set = df.iloc[:int(0.7 * df.shape[0])]
test_set = df.iloc[int(0.7 * df.shape[0]):]

Erstellen einer Random Spalte, um die Signifikanz beim Lasso noch besser zu testen

In [ ]:
train_set['random'] = np.random.normal(0,1,train_set.shape[0])

In [ ]:
train_set['Target_Variable'].hist()

X-Y-Train Test Split

X_train = train_set.drop(['TargetVariable'], axis=1)
y_train = train_set[['TargetVariable']]
X_test = test_set.drop(['TargetVariable'], axis=1)
y_test = test_set[['TargetVariable']]

### Lasso Feature Selection

In [ ]:
lasso_model = Lasso()
# lasso_model.fit(train_set.drop(['TargetVariable'], axis = 1), train_set['TargetVariable'])
lasso_model.fit(X_train, y_train)

In [ ]:
importance = np.abs(coefficients)

In [ ]:
features = train_set.columns.to_list()

In [ ]:
relevant_features_lasso = np.array(features)[importance > 0]

print("Relevante Features:", relevant_features_lasso)

In [ ]:
random_importance_index = features.index('random')
random_importance_value = importance[random_importance_index]
print(random_importance_value)

In [ ]:
relevant_features_lasso = np.array(features)[importance > random_importance_value]

print("Relevante Features:", relevant_features_lasso)

In [ ]:
np.array(features)[importance == 0]

In [ ]:
relevant_features_indices = np.where(importance > random_importance_value)[0]
relevant_importance = importance[relevant_features_indices]

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(relevant_features_lasso, relevant_importance, color='skyblue')
plt.xlabel('Importance (absolute value of coefficients)')
plt.ylabel('Features')
plt.title('Relevant Feature Importance from Lasso Regression')
plt.show()

In [ ]:
X_train = train_set[relevant_features_lasso]
y_train = train_set[['Days_to_Actual_Delivery']]
X_test = test_set[relevant_features_lasso]
y_test = test_set[['Days_to_Actual_Delivery']]

In [ ]:
print("Shape of Train Features: {}".format(X_train.shape))
print("Shape of Test Features: {}".format(X_test.shape))
print("Shape of Train Target: {}".format(y_train.shape))
print("Shape of Test Target: {}".format(y_test.shape))

In [ ]:
plt.figure(figsize=(17, 10))
    
# Plot y_train

train_set[['Order_Creation_Date_W', 'Days_to_Actual_Delivery']].groupby('Order_Creation_Date_W').mean().plot(label='Train')
    
test_set[['Order_Creation_Date_W', 'Days_to_Actual_Delivery']].groupby('Order_Creation_Date_W').mean().plot(label='Test')
    
plt.title('Train and Test Delivery Delay Over Time')
plt.xlabel('Order Creation Week')
plt.ylabel('Delivery Delay')
plt.legend()
plt.show()